In [36]:
# Import Library
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import Sequential

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import  CSVLogger
from tensorflow.keras.callbacks import  ReduceLROnPlateau
from tensorflow.keras.callbacks import  EarlyStopping

In [37]:
# Hyper Parameters
num_classes = 7
img_shape = (224, 224, 3)
batch_size = 32
train_epochs = 50
lr = 1e-3
train_dropout = 0.1

In [38]:
# preprocrssing input
from tensorflow.keras.applications.mobilenet import preprocess_input

In [39]:
# Tranucated MobileNetV1
from tensorflow.keras.applications.mobilenet import MobileNet
input_layer = Input(shape=img_shape)
backbone = MobileNet(include_top = False , weights = "imagenet" , input_shape=(224, 224, 3))
backbone.trainable = False
base_model = tf.keras.Model(backbone.input, backbone.layers[-29].output, name='base_model')

In [40]:
base_model.summary()

Model: "base_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 56, 56, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 821,952 (3.14 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 821,952 (3.14 MB)

In [41]:
# Patch Extraction
Features = Sequential()
Features.add(SeparableConv2D(filters = 512 , kernel_size = 4 , strides = 4 ,padding = "same", activation='relu'))
Features.add(SeparableConv2D(filters = 256 , kernel_size = 2 , strides = 2 ,padding = "valid", activation='relu'))
Features.add(Conv2D(filters = 256, kernel_size=1, strides=1, padding='valid', activation='relu'))

In [42]:
# fully Connected Layer
pre_classification = Sequential()
pre_classification.add(Dense(32, activation='relu'))
pre_classification.add(BatchNormalization())

In [43]:
# Dot Prouduct Self_Attention
self_Attention = Attention(use_scale=True)

In [44]:
# fully connected Layer
output_layer = Dense(num_classes, activation="softmax")

In [45]:
# Building model
from tensorflow.keras.layers import Lambda
inputs = input_layer
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = Features(x)
x = GlobalAveragePooling2D(name='GAP')(x)
x = Dropout(train_dropout)(x)
x = pre_classification(x)
x = tf.keras.layers.Reshape((1, -1))(x)
x = self_Attention([x, x])
x = Lambda(lambda t: tf.squeeze(t, axis=1))(x)
outputs = output_layer(x)
model = tf.keras.Model(inputs, outputs)

In [46]:
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ true_divide_6 (TrueDivide)    │ (None, 224, 224, 3)       │               0 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ subtract_6 (Subtract)         │ (None, 224, 224, 3)       │               0 │ true_divide_6[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ base_model (Functional)       │ (None, 14, 14, 512)       │         821,952 │ subtract_6[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_4 (Sequential)     │ (None, 2, 2, 256)         │         470,016 │ base_model[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ GAP (GlobalAveragePooling2D)  │ (None, 256)               │               0 │ sequential_4[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_6 (Dropout)           │ (None, 256)               │               0 │ GAP[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_5 (Sequential)     │ (None, 32)                │           8,352 │ dropout_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_6 (Reshape)           │ (None, 1, 32)             │               0 │ sequential_5[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_2 (Attention)       │ (None, 1, 32)             │               1 │ reshape_6[0][0],           │
│                               │                           │                 │ reshape_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_2 (Lambda)             │ (None, 32)                │               0 │ attention_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 7)                 │             231 │ lambda_2[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,300,552 (4.96 MB)

 Trainable params: 478,536 (1.83 MB)

 Non-trainable params: 822,016 (3.14 MB)

In [47]:
# read the datset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                             zoom_range = 0.2,
                            shear_range = 0.2,
                            horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory("D:/data/FER2013/train",
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = "categorical")
test_set = test_datagen.flow_from_directory("D:/data/FER2013/train",
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = "categorical")

Found 28709 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [48]:
import os
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir("files")

In [49]:
model_path = os.path.join("files", "model.keras")
csv_path = os.path.join("files", "log.csv")

In [50]:
# Training Procedure
callbacks = [
    ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False)]

In [51]:
# compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr,
                                              global_clipnorm=3.0),
                                              loss='categorical_crossentropy',
                                              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_set,
    epochs=train_epochs,
    validation_data=test_set,
    callbacks=callbacks
)

Epoch 1/50


C:\Users\Mohamed_Abdeldayem\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  9/898 ━━━━━━━━━━━━━━━━━━━━ 1:08:17 5s/step - accuracy: 0.1659 - loss: 1.9431

In [ ]:
#Plot The Training And Testing
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.figure(1)
plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim([0.75,1])
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('plot1')

In [ ]:
# summarize history for loss
plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim([0,15])
plt.legend(['train', 'test'], loc='upper right')
plt.savefig('plot2')